In [81]:
#READ DATA
using CSV, DataFrames

df_nut = CSV.read("data_nutrients.csv", DataFrame, delim = ";")
df_rec = CSV.read("data_recipes.csv", DataFrame, delim = ";")
df_recConst = CSV.read("recipes_const.csv", DataFrame, delim = ";")
#agg_df_recConst = combine(groupby(df_recConst,["recipe"]))


,recipe,need,quantity
,String31,String15,Int64
1,chicken with potatoes,chicken,3
2,chicken with potatoes,potatoes,6
3,beef with rice,beef,8
4,beef with rice,rice,12


In [89]:
#DEFINE SETS AND INDICES

#----------------------------------------------------------------------
#set of days (d)
d = 30
D = [j for j in range(1,d)]

#set of meals (m)
m = 3
M = [j for j in range(1,m)]

#set of nutrients (n)
N = df_nut[:,1]

#set of recipes (r)
R = df_rec[:,1]

#set of people (p)
p = 3
P = [j for j in range(1,p)]

#set of ingredients (i)
#....

#----------------------------------------------------------------------
#indice rmd
rmd = [(l,j,k) for l in R for j in M for k in D]

#indice rmdp
rmdp = [(l,j,k,f) for l in R for j in M for k in D for f in P]

#indice np
np = [(u,f) for u in N for f in P]

#indice nr
nr =[(u,l) for u in N for l in R]

#indice ir
ir = [(row.recipe,row.need) for row in eachrow(df_recConst)]

4-element Vector{Tuple{String31, String15}}:
 ("chicken with potatoes", "chicken")
 ("chicken with potatoes", "potatoes")
 ("beef with rice", "beef")
 ("beef with rice", "rice")